<a href="https://colab.research.google.com/github/DeepikaP2003/CVIP-DATA-SCIENCE-INTERNSHIP-TASK-2/blob/main/Quant_model_implementation_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


# Loading Dataset

In [ ]:
!unzip "/content/drive/MyDrive/Quantum/Dataset_BreastCancer.zip"

Archive:  /content/drive/MyDrive/Quantum/Dataset_BreastCancer.zip
  inflating: Dataset_BUSI_with_GT/benign/benign (1).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (1)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (10).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (10)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (100).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (100)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (100)_mask_1.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (101).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (101)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (102).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (102)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (103).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (103)_mask.png  
  inflating: Dataset_BUSI_with_GT/benign/benign (104).png  
  inflating: Dataset_BUSI_with_GT/benign/benign (104)_mask.png 

In [ ]:
!pip install split-folders

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras import layers
import os

import cv2

from PIL import Image, ImageOps
import scipy.ndimage as ndi

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense,Dropout
from tensorflow.keras.optimizers.legacy import SGD,Adam
import matplotlib.pyplot as plt

from skimage import transform
import splitfolders
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
import seaborn as sns
from scipy.optimize import minimize


import splitfolders

In [ ]:
!pip install --quiet cirq
import cirq as c

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.7/532.7 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.3/69.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.5/596.5 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [ ]:
os.listdir('/content/Dataset_BUSI_with_GT')

['normal', 'benign', 'malignant']

# Data Preprocessing

In [ ]:
import os
import shutil
import random

# Set your dataset directory and the output directory
dataset_directory = "/content/Dataset_BUSI_with_GT"  # Adjust this path if needed
output_directory = "outputdata"

# Define the ratios for splitting
train_ratio = 0.5
validation_ratio = 0.4
test_ratio = 0.1

# Create output directories for train, validation, and test
os.makedirs(os.path.join(output_directory, "train"), exist_ok=True)
os.makedirs(os.path.join(output_directory, "validation"), exist_ok=True)
os.makedirs(os.path.join(output_directory, "test"), exist_ok=True)

# Get subdirectories (classes) in the dataset directory
classes = [d for d in os.listdir(dataset_directory) if os.path.isdir(os.path.join(dataset_directory, d))]

# Loop through each class directory
for cls in classes:
    cls_path = os.path.join(dataset_directory, cls)
    files = [f for f in os.listdir(cls_path) if os.path.isfile(os.path.join(cls_path, f)) and f.endswith(".png")]

    # Randomly shuffle the files
    random.shuffle(files)

    # Calculate the split points based on ratios
    total_files = len(files)
    train_split = int(total_files * train_ratio)
    validation_split = train_split + int(total_files * validation_ratio)

    # Copy files to their respective directories
    for i, file in enumerate(files):
        source_path = os.path.join(cls_path, file)

        if i < train_split:
            destination_directory = "train"
        elif i < validation_split:
            destination_directory = "validation"
        else:
            destination_directory = "test"

        destination_path = os.path.join(output_directory, destination_directory, cls)
        os.makedirs(destination_path, exist_ok=True)

        shutil.copy(source_path, os.path.join(destination_path, file))

print("Dataset split completed.")


Dataset split completed.


In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255)
valid_datagen = ImageDataGenerator(rescale=1/255)
test_datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the image size and batch size
image_size = (4, 4)
batch_size = 100

# Create an ImageDataGenerator for training, validation, and testing
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the datasets from the directories
train_dataset = train_datagen.flow_from_directory(
    '/content/outputdata/train',
    target_size=image_size,
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True
)

valid_dataset = valid_datagen.flow_from_directory(
    '/content/outputdata/validation',
    target_size=image_size,
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True
)

test_dataset = test_datagen.flow_from_directory(
    '/content/outputdata/test',
    target_size=image_size,
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True
)


Found 788 images belonging to 3 classes.
Found 630 images belonging to 3 classes.
Found 160 images belonging to 3 classes.


In [ ]:
# Extract batches of data
X_train, y_train = next(train_dataset)  #fetches the next batch
X_valid, y_valid = next(valid_dataset)
X_test, y_test = next(test_dataset)

# Flatten the images
X_train = X_train.reshape(X_train.shape[0], -1)   #multi-dim to 1D vector
X_valid = X_valid.reshape(X_valid.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Convert labels to one-hot encoding format
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
lb.fit([0, 1, 2])  # Assuming labels are 0, 1, 2 for the three classes
y_train = lb.transform(y_train.argmax(axis=1))
y_valid = lb.transform(y_valid.argmax(axis=1))
y_test = lb.transform(y_test.argmax(axis=1))



# Quantum Circuit Designing

In [ ]:
import cirq as c
import numpy as np

#np.random.seed(42)

def create_quantum_circuit(qubits, params):
    circuit = c.Circuit()
    num_qubits = len(qubits)

    # Layer 1
    circuit.append(c.Moment([c.rx(params[0 * num_qubits + i])(q) for i, q in enumerate(qubits)]))
    circuit.append(c.Moment([c.ry(params[1 * num_qubits + i])(q) for i, q in enumerate(qubits)]))
    for i in range(num_qubits - 1):
        circuit.append(c.Moment(c.CNOT(qubits[i], qubits[i + 1])))
    circuit.append(c.Moment(c.CNOT(qubits[15], qubits[0])))

    # Barrier (Empty Moment)
    circuit.append(c.Moment())

    # Layer 2
    circuit.append(c.Moment([c.rx(params[2 * num_qubits + i])(q) for i, q in enumerate(qubits)]))
    circuit.append(c.Moment([c.ry(params[3 * num_qubits + i])(q) for i, q in enumerate(qubits)]))
    for i in range(num_qubits - 1):
        circuit.append(c.Moment(c.CNOT(qubits[i], qubits[i + 1])))
    circuit.append(c.Moment(c.CNOT(qubits[15], qubits[0])))

    # Barrier (Empty Moment)
    circuit.append(c.Moment())

    # Layer 3
    circuit.append(c.Moment([c.rx(params[4 * num_qubits + i])(q) for i, q in enumerate(qubits)]))
    circuit.append(c.Moment([c.ry(params[5 * num_qubits + i])(q) for i, q in enumerate(qubits)]))
    for i in range(num_qubits - 1):
        circuit.append(c.Moment(c.CNOT(qubits[i], qubits[i + 1])))
    circuit.append(c.Moment(c.CNOT(qubits[15], qubits[0])))

    # Barrier (Empty Moment)
    circuit.append(c.Moment())

    # Measurement
    measure = c.Moment([c.measure(qubit, key=f'qubit_{i}') for i, qubit in enumerate(qubits)])
    circuit.append(measure)

    return circuit

# 16 qubits for a 4x4 image
num_qubits = 16
qubits = [c.LineQubit(i) for i in range(num_qubits)]

# Adjust the number of parameters to match the number of gates
# 6 layers (3 layers with RX and RY each) * 16 qubits = 96 parameters
# Total parameters = num_qubits * number of layers * 2 (since each qubit has an RX and RY gate per layer)
params = np.random.rand(96)  # Random initialization of 96 parameters

# Initialize parameters with fixed small random values
#params = np.random.uniform(low=-0.1, high=0.1, size=num_qubits * 6)
# Create the circuit
circuit = create_quantum_circuit(qubits, params)
print(circuit)



0: ────Rx(0.097π)───Ry(0.229π)───@───────────────────────────────────────────────────────────X───────Rx(0.272π)───Ry(0.253π)───@───────────────────────────────────────────────────────────X───────Rx(0.228π)───Ry(0.221π)───@───────────────────────────────────────────────────────────X───────M('qubit_0')────
                                 │                                                           │                                 │                                                           │                                 │                                                           │
1: ────Rx(0.187π)───Ry(0.276π)───X───@───────────────────────────────────────────────────────┼───────Rx(0.229π)───Ry(0.152π)───X───@───────────────────────────────────────────────────────┼───────Rx(0.119π)───Ry(0.165π)───X───@───────────────────────────────────────────────────────┼───────M('qubit_1')────
                                     │                                                       │           

In [ ]:
# Quantum-classical hybrid function that returns predictions for each class
def quantum_model(params, data_point, num_classes=3):
    num_qubits = len(data_point)
    qubits = c.GridQubit.rect(1, num_qubits)

    # Create separate circuits for each class and get expectation values
    predictions = []
    for _ in range(num_classes):
        circuit = create_quantum_circuit(qubits, params)
        simulator = c.Simulator()
        result = simulator.simulate(circuit)
        predictions.append(np.mean(result.final_state_vector.real))

    # Convert predictions to a probability distribution (softmax)
    predictions = np.exp(predictions)
    predictions /= np.sum(predictions)
    return predictions

In [ ]:
# Define loss function using categorical cross-entropy
def loss(params, x_train_batch, y_train_batch, num_classes=3):
    total_loss = 0
    for i, x in enumerate(x_train_batch):
        predictions = quantum_model(params, x, num_classes=num_classes)
        y_true = y_train_batch[i]
        # Compute categorical cross-entropy
        loss_value = -np.sum(y_true * np.log(predictions))
        total_loss += loss_value
    return total_loss / len(x_train_batch)

In [ ]:

# Training loop with COBYLA optimizer
def train(x_train, y_train, num_qubits, num_classes=3):
    # Initialize variational parameters
    init_params = np.random.uniform(0, np.pi, num_qubits * 2)

    # Use the COBYLA optimizer
    result = minimize(fun=loss, x0=init_params, args=(x_train, y_train, num_classes), method='COBYLA')

    return result.x  # optimized parameters

In [ ]:
# Accuracy calculation
def calculate_accuracy(params, x_test, y_test, num_classes=3):
    correct_predictions = 0
    for i, x in enumerate(x_test):
        predictions = quantum_model(params, x, num_classes=num_classes)
        predicted_label = np.argmax(predictions)  # Get the index of the highest probability
        true_label = np.argmax(y_test[i])  # Get the true label index from the one-hot encoded label
        if predicted_label == true_label:
            correct_predictions += 1
    accuracy = correct_predictions / len(x_test)
    return accuracy

# Running epochs

# Simple Training

In [ ]:

# Assuming num_qubits equals the size of the i/p data
#optimized_params = train(x_train_batch, y_train_batch, num_qubits=num_qubits)
def train(x_train, y_train, num_qubits):
    # Assuming the circuit needs num_qubits * 6 parameters
    optimized_params = np.random.rand(num_qubits * 6)  # Adjusted based on circuit design
    return optimized_params
# Example training call
batch_size = 100
x_train_batch = X_train[:batch_size]
y_train_batch = y_train[:batch_size]

optimized_params = train(x_train_batch, y_train_batch, num_qubits=num_qubits)

# Number of qubits = number of features (flattened image size)
num_qubits = image_size[0] * image_size[1]
#print("Optimized Parameters:", optimized_params)

In [ ]:
# Calculate accuracy on test set
accuracy = calculate_accuracy(optimized_params, X_test[:batch_size], y_test[:batch_size])
print("Model Accuracy: {:.2f}%".format(accuracy * 100))

Model Accuracy: 37.00%


# Using Class

# #1

In [ ]:
import numpy as np

from scipy.optimize import minimize

class QuantumModel:
    def __init__(self, num_qubits, num_layers=3, num_classes=3):
        self.num_qubits = num_qubits
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.params = np.random.uniform(low=-0.1, high=0.1, size=num_qubits * num_layers * 2)

    def create_quantum_circuit(self, qubits, params):
        circuit = c.Circuit()
        for layer in range(self.num_layers):
            # RX and RY rotations
            circuit.append(c.Moment([c.rx(params[layer * 2 * self.num_qubits + i])(q) for i, q in enumerate(qubits)]))
            circuit.append(c.Moment([c.ry(params[layer * 2 * self.num_qubits + self.num_qubits + i])(q) for i, q in enumerate(qubits)]))
            # CNOT chain
            for i in range(self.num_qubits - 1):
                circuit.append(c.Moment(c.CNOT(qubits[i], qubits[i + 1])))
            circuit.append(c.Moment(c.CNOT(qubits[-1], qubits[0])))
            # Barrier (Empty Moment)
            circuit.append(c.Moment())

        # Measurement
        measure = c.Moment([c.measure(qubit, key=f'qubit_{i}') for i, qubit in enumerate(qubits)])
        circuit.append(measure)

        return circuit

    def loss(self, params, x_train, y_train):
        total_loss = 0.0
        for i, x in enumerate(x_train):
            predictions = self.quantum_model(params, x)
            y_true = y_train[i]
            # Compute categorical cross-entropy loss
            loss_value = -np.sum(y_true * np.log(predictions + 1e-10))  # Added epsilon to avoid log(0)
            total_loss += loss_value
        return total_loss / len(x_train)

    def calculate_accuracy(self, params, x_test, y_test):
        correct_predictions = 0
        for i, x in enumerate(x_test):
            predictions = self.quantum_model(params, x)
            predicted_label = np.argmax(predictions)
            true_label = np.argmax(y_test[i])
            if predicted_label == true_label:
                correct_predictions += 1
        accuracy = correct_predictions / len(x_test)
        return accuracy

    def compile(self, optimizer='COBYLA', loss=None, metrics=None):
        self.optimizer = optimizer
        if loss:
            self.loss = loss
        self.metrics = metrics

    def fit(self, x_train, y_train, epochs=20, validation_data=None):
        for epoch in range(epochs):
            # Optimize parameters using the specified optimizer
            result = minimize(fun=self.loss, x0=self.params, args=(x_train, y_train), method=self.optimizer)
            self.params = result.x  # Update the parameters

            # Compute loss
            train_loss = self.loss(self.params, x_train, y_train)

            # Compute accuracy on training data
            train_accuracy = self.calculate_accuracy(self.params, x_train, y_train)

            # If validation data is provided, compute validation accuracy
            if validation_data:
                x_val, y_val = validation_data
                val_accuracy = self.calculate_accuracy(self.params, x_val, y_val)
                print(f'Epoch {epoch+1}/{epochs}: Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}')
            else:
                print(f'Epoch {epoch+1}/{epochs}: Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')

    def quantum_model(self, params, x):
        # Define the quantum model logic
        num_qubits = len(x)  # Assume `x` is a vector representing input data
        qubits = [c.LineQubit(i) for i in range(num_qubits)]

        # Create the circuit and get the state vector
        circuit = self.create_quantum_circuit(qubits, params)
        simulator = c.Simulator()
        result = simulator.simulate(circuit)
        final_state = result.final_state_vector

        # Convert the state vector to a probability distribution for classification
        probabilities = np.abs(final_state)**2
        return probabilities[:self.num_classes]

# Example usage
num_qubits = 16  # For a 4x4 image
model = QuantumModel(num_qubits=num_qubits, num_layers=3, num_classes=3)

# Compile the model with the optimizer and loss function
model.compile(optimizer='L-BFGS-B', loss=model.loss)

# Example training data
batch_size = 30
x_train_batch = X_train[:batch_size]
y_train_batch = y_train[:batch_size]
x_val_batch = X_valid[:batch_size]
y_val_batch = y_valid[:batch_size]

# Train the model with 20 epochs
model.fit(x_train_batch, y_train_batch, epochs=5, validation_data=(x_val_batch, y_val_batch))


# #2

In [ ]:
import numpy as np
from scipy.optimize import minimize

class QuantumModel:
    def __init__(self, num_qubits, num_layers=2, num_classes=3):
        self.num_qubits = num_qubits
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.params = np.random.uniform(low=-0.5, high=0.5, size=num_qubits * num_layers * 2)

    def create_quantum_circuit(self, qubits, params):
        circuit = c.Circuit()
        for layer in range(self.num_layers):
            # RX and RY rotations
            circuit.append(c.Moment([c.rx(params[layer * 2 * self.num_qubits + i])(q) for i, q in enumerate(qubits)]))
            circuit.append(c.Moment([c.ry(params[layer * 2 * self.num_qubits + self.num_qubits + i])(q) for i, q in enumerate(qubits)]))
            # CNOT chain
            for i in range(self.num_qubits - 1):
                circuit.append(c.Moment(c.CNOT(qubits[i], qubits[i + 1])))
            circuit.append(c.Moment(c.CNOT(qubits[-1], qubits[0])))
            # Barrier (Empty Moment)
            circuit.append(c.Moment())

        # Measurement
        measure = c.Moment([c.measure(qubit, key=f'qubit_{i}') for i, qubit in enumerate(qubits)])
        circuit.append(measure)

        return circuit

    def loss(self, params, x_train, y_train):
        total_loss = 0.0
        for i, x in enumerate(x_train):
            predictions = self.quantum_model(params, x)
            y_true = y_train[i]
            # Compute categorical cross-entropy loss
            loss_value = -np.sum(y_true * np.log(predictions + 1e-10))  # Added epsilon to avoid log(0)
            total_loss += loss_value
        return total_loss / len(x_train)

    def calculate_accuracy(self, params, x_test, y_test):
        correct_predictions = 0
        for i, x in enumerate(x_test):
            predictions = self.quantum_model(params, x)
            predicted_label = np.argmax(predictions)
            true_label = np.argmax(y_test[i])
            if predicted_label == true_label:
                correct_predictions += 1
        accuracy = correct_predictions / len(x_test)
        return accuracy

    def compile(self, optimizer='L-BFGS-B', loss=None, metrics=None):  # Use L-BFGS-B optimizer
        self.optimizer = optimizer
        if loss:
            self.loss = loss
        self.metrics = metrics

    def fit(self, x_train, y_train, epochs=10, validation_data=None):  # Reduced to 10 epochs
        for epoch in range(epochs):
            # Optimize parameters using the specified optimizer
            result = minimize(fun=self.loss, x0=self.params, args=(x_train, y_train), method=self.optimizer)
            self.params = result.x  # Update the parameters

            # Compute loss
            train_loss = self.loss(self.params, x_train, y_train)

            # Compute accuracy on training data
            train_accuracy = self.calculate_accuracy(self.params, x_train, y_train)

            # If validation data is provided, compute validation accuracy
            if validation_data:
                x_val, y_val = validation_data
                val_accuracy = self.calculate_accuracy(self.params, x_val, y_val)
                print(f'Epoch {epoch+1}/{epochs}: Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}')
            else:
                print(f'Epoch {epoch+1}/{epochs}: Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')

    def quantum_model(self, params, x):
        # Define the quantum model logic
        num_qubits = len(x)  # Assume `x` is a vector representing input data
        qubits = [c.LineQubit(i) for i in range(num_qubits)]

        # Create the circuit and get the state vector
        circuit = self.create_quantum_circuit(qubits, params)
        simulator = c.Simulator()
        result = simulator.simulate(circuit)
        final_state = result.final_state_vector

        # Convert the state vector to a probability distribution for classification
        probabilities = np.abs(final_state)**2
        return probabilities[:self.num_classes]

# Example usage
num_qubits = 16  # For a 4x4 image
model = QuantumModel(num_qubits=num_qubits, num_layers=2, num_classes=3)  # Reduced to 2 layers

# Compile the model with the optimizer and loss function
model.compile(optimizer='L-BFGS-B', loss=model.loss)  # Using L-BFGS-B optimizer

# Example training data
batch_size = 20  # Smaller batch size to reduce memory usage
x_train_batch = X_train[:batch_size]
y_train_batch = y_train[:batch_size]
x_val_batch = X_valid[:batch_size]
y_val_batch = y_valid[:batch_size]

# Train the model with 10 epochs
model.fit(x_train_batch, y_train_batch, epochs=5, validation_data=(x_val_batch, y_val_batch))


Epoch 1/5: Loss: 18.4207, Train Accuracy: 0.5500, Validation Accuracy: 0.6000
Epoch 2/5: Loss: 20.7233, Train Accuracy: 0.5500, Validation Accuracy: 0.6000
Epoch 3/5: Loss: 20.7233, Train Accuracy: 0.5500, Validation Accuracy: 0.6000
Epoch 4/5: Loss: 20.7233, Train Accuracy: 0.6000, Validation Accuracy: 0.6000
Epoch 5/5: Loss: 17.2694, Train Accuracy: 0.5500, Validation Accuracy: 0.6000


In [ ]:
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import f1_score

class QuantumModel:
    def __init__(self, num_qubits, num_layers=2, num_classes=3):
        self.num_qubits = num_qubits
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.params = np.random.uniform(low=-0.5, high=0.5, size=num_qubits * num_layers * 2)

    def create_quantum_circuit(self, qubits, params):
        circuit = c.Circuit()
        for layer in range(self.num_layers):
            # RX and RY rotations
            circuit.append(c.Moment([c.rx(params[layer * 2 * self.num_qubits + i])(q) for i, q in enumerate(qubits)]))
            circuit.append(c.Moment([c.ry(params[layer * 2 * self.num_qubits + self.num_qubits + i])(q) for i, q in enumerate(qubits)]))
            # CNOT chain
            for i in range(self.num_qubits - 1):
                circuit.append(c.Moment(c.CNOT(qubits[i], qubits[i + 1])))
            circuit.append(c.Moment(c.CNOT(qubits[-1], qubits[0])))
            # Barrier (Empty Moment)
            circuit.append(c.Moment())

        # Measurement
        measure = c.Moment([c.measure(qubit, key=f'qubit_{i}') for i, qubit in enumerate(qubits)])
        circuit.append(measure)

        return circuit

    def loss(self, params, x_train, y_train):
        total_loss = 0.0
        for i, x in enumerate(x_train):
            predictions = self.quantum_model(params, x)
            y_true = y_train[i]
            # Compute categorical cross-entropy loss
            loss_value = -np.sum(y_true * np.log(predictions + 1e-10))  # Added epsilon to avoid log(0)
            total_loss += loss_value
        return total_loss / len(x_train)

    def calculate_f1_score(self, params, x_test, y_test):
        y_pred = []
        y_true = []
        for i, x in enumerate(x_test):
            predictions = self.quantum_model(params, x)
            predicted_label = np.argmax(predictions)
            y_pred.append(predicted_label)
            y_true.append(np.argmax(y_test[i]))

        f1 = f1_score(y_true, y_pred, average='macro')
        return f1

    def compile(self, optimizer='COBYLA', loss=None, metrics=None):
        self.optimizer = optimizer
        if loss:
            self.loss = loss
        self.metrics = metrics

    def fit(self, x_train, y_train, epochs=10, validation_data=None):
        for epoch in range(epochs):
            # Optimize parameters using the specified optimizer
            result = minimize(fun=self.loss, x0=self.params, args=(x_train, y_train), method=self.optimizer)
            self.params = result.x  # Update the parameters

            # Compute loss
            train_loss = self.loss(self.params, x_train, y_train)

            # Compute F1 Score on training data
            train_f1 = self.calculate_f1_score(self.params, x_train, y_train)

            # If validation data is provided, compute validation F1 Score
            if validation_data:
                x_val, y_val = validation_data
                val_f1 = self.calculate_f1_score(self.params, x_val, y_val)
                print(f'Epoch {epoch+1}/{epochs}: Loss: {train_loss:.4f}, Train F1 Score: {train_f1:.4f}, Validation F1 Score: {val_f1:.4f}')
            else:
                print(f'Epoch {epoch+1}/{epochs}: Loss: {train_loss:.4f}, Train F1 Score: {train_f1:.4f}')

    def quantum_model(self, params, x):
        # Define the quantum model logic
        num_qubits = len(x)  # Assume `x` is a vector representing input data
        qubits = [c.LineQubit(i) for i in range(num_qubits)]

        # Create the circuit and get the state vector
        circuit = self.create_quantum_circuit(qubits, params)
        simulator = c.Simulator()
        result = simulator.simulate(circuit)
        final_state = result.final_state_vector

        # Convert the state vector to a probability distribution for classification
        probabilities = np.abs(final_state)**2
        return probabilities[:self.num_classes]

# Example usage
num_qubits = 16  # For a 4x4 image
model = QuantumModel(num_qubits=num_qubits, num_layers=2, num_classes=3)  # Reduced to 2 layers

# Compile the model with the optimizer and loss function
model.compile(optimizer='COBYLA', loss=model.loss)  # Using COBYLA optimizer

# Example training data
batch_size = 30  # Smaller batch size to reduce memory usage
x_train_batch = X_train[:batch_size]
y_train_batch = y_train[:batch_size]
x_val_batch = X_valid[:batch_size]
y_val_batch = y_valid[:batch_size]

# Train the model with 10 epochs
model.fit(x_train_batch, y_train_batch, epochs=10, validation_data=(x_val_batch, y_val_batch))


Epoch 1/10: Loss: 19.9557, Train F1 Score: 0.2821, Validation F1 Score: 0.2319
Epoch 2/10: Loss: 19.1882, Train F1 Score: 0.2821, Validation F1 Score: 0.2319
Epoch 3/10: Loss: 21.4908, Train F1 Score: 0.2821, Validation F1 Score: 0.2319
Epoch 4/10: Loss: 23.0259, Train F1 Score: 0.2821, Validation F1 Score: 0.2319
Epoch 5/10: Loss: 23.0259, Train F1 Score: 0.2821, Validation F1 Score: 0.2319
Epoch 6/10: Loss: 23.0259, Train F1 Score: 0.2821, Validation F1 Score: 0.2319
Epoch 7/10: Loss: 23.0259, Train F1 Score: 0.2821, Validation F1 Score: 0.2319
Epoch 8/10: Loss: 23.0259, Train F1 Score: 0.2821, Validation F1 Score: 0.2319
Epoch 9/10: Loss: 23.0259, Train F1 Score: 0.2821, Validation F1 Score: 0.2319
Epoch 10/10: Loss: 23.0259, Train F1 Score: 0.2821, Validation F1 Score: 0.2319


# Mini batches

In [ ]:
import numpy as np
from scipy.optimize import minimize

class QuantumModel:
    def __init__(self, num_qubits, num_layers=2, num_classes=3):
        self.num_qubits = num_qubits
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.params = np.random.uniform(low=-0.5, high=0.5, size=num_qubits * num_layers * 2)

    def create_quantum_circuit(self, qubits, params):
        circuit = c.Circuit()
        for layer in range(self.num_layers):
            # RX and RY rotations
            circuit.append(c.Moment([c.rx(params[layer * 2 * self.num_qubits + i])(q) for i, q in enumerate(qubits)]))
            circuit.append(c.Moment([c.ry(params[layer * 2 * self.num_qubits + self.num_qubits + i])(q) for i, q in enumerate(qubits)]))
            # CNOT chain
            for i in range(self.num_qubits - 1):
                circuit.append(c.Moment(c.CNOT(qubits[i], qubits[i + 1])))
            circuit.append(c.Moment(c.CNOT(qubits[-1], qubits[0])))
            # Barrier (Empty Moment)
            circuit.append(c.Moment())

        # Measurement
        measure = c.Moment([c.measure(qubit, key=f'qubit_{i}') for i, qubit in enumerate(qubits)])
        circuit.append(measure)

        return circuit

    def loss(self, params, x_train, y_train):
        total_loss = 0.0
        for i, x in enumerate(x_train):
            predictions = self.quantum_model(params, x)
            y_true = y_train[i]
            # Compute categorical cross-entropy loss
            loss_value = -np.sum(y_true * np.log(predictions + 1e-10))  # Added epsilon to avoid log(0)
            total_loss += loss_value
        return total_loss / len(x_train)

    def calculate_accuracy(self, params, x_test, y_test):
        correct_predictions = 0
        for i, x in enumerate(x_test):
            predictions = self.quantum_model(params, x)
            predicted_label = np.argmax(predictions)
            true_label = np.argmax(y_test[i])
            if predicted_label == true_label:
                correct_predictions += 1
        accuracy = correct_predictions / len(x_test)
        return accuracy

    def compile(self, optimizer='L-BFGS-B', loss=None, metrics=None):
        self.optimizer = optimizer
        if loss:
            self.loss = loss
        self.metrics = metrics

    def fit(self, x_train, y_train, epochs=10, validation_data=None, batch_size=20):
        num_samples = len(x_train)
        for epoch in range(epochs):
            # Shuffle the data at the beginning of each epoch
            indices = np.arange(num_samples)
            np.random.shuffle(indices)
            x_train = x_train[indices]
            y_train = y_train[indices]

            for start_idx in range(0, num_samples, batch_size):
                end_idx = min(start_idx + batch_size, num_samples)
                x_batch = x_train[start_idx:end_idx]
                y_batch = y_train[start_idx:end_idx]

                # Optimize parameters using the specified optimizer for the current mini-batch
                result = minimize(fun=self.loss, x0=self.params, args=(x_batch, y_batch), method=self.optimizer)
                self.params = result.x  # Update the parameters

            # Compute loss and accuracy on the full training set after the epoch
            train_loss = self.loss(self.params, x_train, y_train)
            train_accuracy = self.calculate_accuracy(self.params, x_train, y_train)

            # If validation data is provided, compute validation accuracy
            if validation_data:
                x_val, y_val = validation_data
                val_accuracy = self.calculate_accuracy(self.params, x_val, y_val)
                print(f'Epoch {epoch+1}/{epochs}: Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}')
            else:
                print(f'Epoch {epoch+1}/{epochs}: Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')

    def quantum_model(self, params, x):
        # Define the quantum model logic
        num_qubits = len(x)  # Assume `x` is a vector representing input data
        qubits = [c.LineQubit(i) for i in range(num_qubits)]

        # Create the circuit and get the state vector
        circuit = self.create_quantum_circuit(qubits, params)
        simulator = c.Simulator()
        result = simulator.simulate(circuit)
        final_state = result.final_state_vector

        # Convert the state vector to a probability distribution for classification
        probabilities = np.abs(final_state)**2
        return probabilities[:self.num_classes]

# Example usage
num_qubits = 16  # For a 4x4 image
model = QuantumModel(num_qubits=num_qubits, num_layers=2, num_classes=3)  # Reduced to 2 layers

# Compile the model with the optimizer and loss function
model.compile(optimizer='L-BFGS-B', loss=model.loss)  # Using L-BFGS-B optimizer

# Example training data
batch_size = 20  # Smaller batch size to reduce memory usage
x_train_batch = X_train[:batch_size]
y_train_batch = y_train[:batch_size]
x_val_batch = X_valid[:batch_size]
y_val_batch = y_valid[:batch_size]

# Train the model with 10 epochs
model.fit(x_train_batch, y_train_batch, epochs=10, validation_data=(x_val_batch, y_val_batch), batch_size=batch_size)


Epoch 1/10: Loss: 20.7233, Train Accuracy: 0.3500, Validation Accuracy: 0.1500


KeyboardInterrupt: 

#Implemet PCA

In [ ]:
import numpy as np
import cirq as c
from scipy.optimize import minimize
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical

class QuantumModel:
    def __init__(self, num_qubits, num_layers=2, num_classes=3, learning_rate=0.001):
        self.num_qubits = num_qubits
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.params = np.random.uniform(low=-0.5, high=0.5, size=num_qubits * num_layers * 2)
        self.optimizer = Adam(learning_rate=self.learning_rate)
        self.loss_fn = CategoricalCrossentropy()

    def create_quantum_circuit(self, qubits, params):
        circuit = c.Circuit()
        for layer in range(self.num_layers):
            # RX and RY rotations
            circuit.append(c.Moment([c.rx(params[layer * 2 * self.num_qubits + i])(q) for i, q in enumerate(qubits)]))
            circuit.append(c.Moment([c.ry(params[layer * 2 * self.num_qubits + self.num_qubits + i])(q) for i, q in enumerate(qubits)]))
            # CNOT chain
            for i in range(self.num_qubits - 1):
                circuit.append(c.Moment(c.CNOT(qubits[i], qubits[i + 1])))
            circuit.append(c.Moment(c.CNOT(qubits[-1], qubits[0])))
            # Barrier (Empty Moment)
            circuit.append(c.Moment())

        # Measurement
        measure = c.Moment([c.measure(qubit, key=f'qubit_{i}') for i, qubit in enumerate(qubits)])
        circuit.append(measure)

        return circuit

    def quantum_model(self, params, x):
        num_qubits = len(x)  # Assume `x` is a vector representing input data
        qubits = [c.LineQubit(i) for i in range(num_qubits)]

        # Create the circuit and get the state vector
        circuit = self.create_quantum_circuit(qubits, params)
        simulator = c.Simulator()
        try:
            result = simulator.simulate(circuit)
        except Exception as e:
            print(f"Simulation failed: {e}")
            raise

        final_state = result.final_state_vector

        # Check for NaN values in the final state vector
        if np.any(np.isnan(final_state)):
            raise ValueError("Final state vector contains NaN values")

        # Convert the state vector to a probability distribution for classification
        probabilities = np.abs(final_state)**2

        # Check for NaN values in the probabilities
        if np.any(np.isnan(probabilities)):
            raise ValueError("Probability distribution contains NaN values")

        return probabilities[:self.num_classes]



    def compute_loss(self, params, x_batch, y_batch):
        total_loss = 0.0
        num_samples = len(x_batch)
        for i in range(num_samples):
            x = x_batch[i]
            y_true = y_batch[i]
            predictions = self.quantum_model(params, x)
            predictions = np.array(predictions, dtype=np.float32)
            y_true = np.array(y_true, dtype=np.float32)
            loss_value = self.loss_fn(y_true, predictions).numpy()
            total_loss += loss_value
        return total_loss / num_samples

    def calculate_accuracy(self, x_test, y_test):
        correct_predictions = 0
        num_samples = len(x_test)
        for i in range(num_samples):
            x = x_test[i]
            y_true = y_test[i]
            predictions = self.quantum_model(self.params, x)
            predicted_label = np.argmax(predictions)
            true_label = np.argmax(y_true)
            if predicted_label == true_label:
                correct_predictions += 1

        accuracy = correct_predictions / num_samples
        return accuracy

    def fit(self, x_train, y_train, epochs=10, validation_data=None, batch_size=20):
        num_samples = len(x_train)
        for epoch in range(epochs):
            # Shuffle the data at the beginning of each epoch
            indices = np.arange(num_samples)
            np.random.shuffle(indices)
            x_train = x_train[indices]
            y_train = y_train[indices]

            for start_idx in range(0, num_samples, batch_size):
                end_idx = min(start_idx + batch_size, num_samples)
                x_batch = x_train[start_idx:end_idx]
                y_batch = y_train[start_idx:end_idx]

                # Optimize parameters using the specified optimizer for the current mini-batch
                result = minimize(fun=lambda p: self.compute_loss(p, x_batch, y_batch),
                                  x0=self.params,
                                  method='L-BFGS-B')
                self.params = result.x  # Update the parameters

            # Compute loss and accuracy on the full training set after the epoch
            train_loss = self.compute_loss(self.params, x_train, y_train)
            train_accuracy = self.calculate_accuracy(x_train, y_train)

            # If validation data is provided, compute validation accuracy
            if validation_data:
                x_val, y_val = validation_data
                val_accuracy = self.calculate_accuracy(x_val, y_val)
                print(f'Epoch {epoch+1}/{epochs}: Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}')
            else:
                print(f'Epoch {epoch+1}/{epochs}: Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')

# Example usage
num_qubits = 16  # For a 4x4 image
model = QuantumModel(num_qubits=num_qubits, num_layers=2, num_classes=3, learning_rate=0.001)  # Adjust learning rate

# Example training data
batch_size = 20  # Smaller batch size to reduce memory usage
X_train = np.random.rand(100, num_qubits).astype(np.float32)  # 100 samples with num_qubits features
y_train = to_categorical(np.random.choice(3, 100), num_classes=3)  # Random one-hot encoded labels for 3 classes
X_valid = np.random.rand(20, num_qubits).astype(np.float32)  # 20 samples for validation
y_valid = to_categorical(np.random.choice(3, 20), num_classes=3)  # Random one-hot encoded labels for validation

# Train the model with 10 epochs
model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), batch_size=batch_size)


Simulation failed: probabilities contain NaN


ValueError: probabilities contain NaN